# Setup Inicial

Instalar o wget caso não possua no seu computador, ou baixar o dataset direto pelo link abaixo

In [ ]:
!wget -q --show-progress http://cin.ufpe.br/~llm2/newAnime.zip -O newAnime.zip
!unzip -o newAnime.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from tqdm import tqdm
from scipy import stats

# Load dos dados


Os dados foram coletados utilizando a API Jikan que acessa o site MyAnimeList

Foi feito a iteração pelos id's disponiveis nesse [dataset](https://www.kaggle.com/marlesson/myanimelist-dataset-animes-profiles-reviews), realizado na data 24/9.

Script disponivel [aqui](https://github.com/lionliu/trending-mal/blob/master/scraping/get_updated_data.py)

In [ ]:
animeList = pd.read_csv("./newAnime.csv")
animeList = animeList.drop_duplicates(subset = ['title'])

In [ ]:
len(animeList)

# Pre-processamento

In [ ]:
animeList.head()

In [ ]:
animeList.describe()

In [ ]:
animeList.dtypes

In [ ]:
animeList.corr()

Remoção das seguintes colunas:


*   mal_id: ID do anime, que não agregaria a analise de dados
*   rank: ordenamento por score dos animes
*   popularity: ordenamento por quantidade de membros



In [ ]:
animeList = animeList.drop(['mal_id', 'rank', 'popularity'], axis=1)

In [ ]:
# animeList[(animeList['genres'] == '[]') & (animeList['studios'] == '[]')]
# 0,03% dos dados, e também, pode não conter source, nem score, e nem rank, e alguns sequer episodios, com rápida pesquisa, aparentam ser curta metragens
print(len(animeList[(animeList['genres'] == '[]') & (animeList['studios'] == '[]')])/len(animeList))
animeList = animeList[(animeList['genres'] != '[]') | (animeList['studios'] != '[]')]

In [ ]:
animeList = animeList.replace({'genres':'[]','studios':'[]'},"['Unknown']")

In [ ]:
# Criar uma coluna a partir da coluna de generos de string para lista
def string_to_list(s):
    return s.strip('][').split(', ')
  
def remove_quotes(s):
    return list(map(lambda a: a.strip('\''), s))

genreList = animeList['genres'].apply(string_to_list).apply(remove_quotes)
studioList = animeList['studios'].apply(string_to_list).apply(remove_quotes)
animeList['studiosList'] = animeList['studios'].apply(string_to_list).apply(remove_quotes)
animeList['genresList'] = animeList['genres'].apply(string_to_list).apply(remove_quotes)

In [ ]:
import itertools
# Gerar uma lista de todos os generos de animes
genreListArr = sorted(set(itertools.chain.from_iterable(genreList)))
studioListArr = sorted(set(itertools.chain.from_iterable(studioList)))
print(genreListArr)
# print(studioListArr)
print("\nQuantidade de gêneros : " + str(len(genreListArr)) + "\nQuantidade de estúdios: " + str(len(studioListArr)))

In [ ]:
# Relação de nulos sobre o total
print(animeList.isna().sum() / len(animeList))
print(animeList.isna().sum())

Mesmo que os scores faltantes tenham uma proporção consideravel de 27%, decidimos tirar pois já temos uma quantidade consideravel de dados, e também é provavél que os animes com scores faltando serem os mais desconhecidos, logo não seriam interessantes para a nossa análise.

In [ ]:
animeList = animeList[animeList['score'].notnull()]

In [ ]:
animeList = animeList[animeList['episodes'].notnull()]

Vamos cortar os animes por 6 quantis de membros, para assim podermos atribuir uma categoria a cada quartil com um rank e fazer uma análise mais na frente.

In [ ]:
animeList['categoryByMembers'] = pd.qcut(animeList['members'], 6)
animeList['categoryByMembers'] = animeList['categoryByMembers'].astype('category')
animeList['categoryByMembers'].value_counts()

In [ ]:
animeList['categoryByMembers'].cat.categories = ['Iron', 'Bronze', 'Silver', 'Gold','Platinum', 'Diamond']
animeList['categoryByMembers'].value_counts()

Fazer uma coluna apenas com o primeiro estúdio da lista  para poder definir-la como categoria.

In [ ]:
len(animeList[animeList.studiosList.str.len() > 1])

In [ ]:
animeList['studio'] = animeList.studiosList.apply(lambda x: x[0])
animeList['studio'] = animeList.studio.astype('category')
animeList['studioCode'] = animeList.studio.cat.codes

In [ ]:
animeList['studioCode']

### Remoção de outliers e normalização

#### Univariada

In [ ]:
# Necessidade de retirar outliers
animeList['members'].plot.box()

In [ ]:
animeList['members_log'] = np.log10(animeList['members'])
print(animeList['members_log'].mad())
animeList['members_log'].hist()

In [ ]:
animeList['members_log'].plot.box()

Os outliers pelo método Z-score robusto com um limiar de 3 são os animes mais famosos, logo sua quantidade de membros é normal para a sua popularidade, não havendo a necessidade de remove-los

In [ ]:
animeList[abs(animeList['members_log']-animeList['members_log'].median()) / animeList['members_log'].mad() > 3]

In [ ]:
q1 = animeList['members_log'].quantile(q=0.25)
q3 = animeList['members_log'].quantile(q=0.75)
# Intervalo interquartil
iqr = q3 - q1
print(iqr)

Há 2 outliers pelo método de Tukey, mas não iremos remove-los pelo mesmo motivo acima

In [ ]:
print(len(animeList[animeList['members_log'] < (q1-1.5*iqr)]))
print(len(animeList[animeList['members_log'] > (q3+1.5*iqr)]))

In [ ]:
animeList[animeList['members_log'] > (q3+1.5*iqr)]

#### Bivariada

Gerar uma nova coluna com a razão de favoritos por membros, pois todos que favoritaram são membros, mas nem todos os membros favoritaram.

In [ ]:
# animeList.plot.scatter('members', 'favorites')
sns.scatterplot(x=animeList['members'], y=animeList['favorites'])

In [ ]:
animeList['favorite_per_member'] = animeList['favorites'] / animeList['members']

In [ ]:
animeList['favorite_per_member'].describe()

In [ ]:
animeList['favorite_per_member'].hist()

In [ ]:
animeList['favorite_per_member'] = np.log10(animeList['favorite_per_member'])

In [ ]:
animeList['favorite_per_member'].plot.box()

Não há outliers pelo z-score robusto

In [ ]:
animeList[abs(animeList['favorite_per_member']-animeList['favorite_per_member'].median())/animeList['favorite_per_member'].mad() > 3]

In [ ]:
q1 = animeList['favorite_per_member'].quantile(q=0.25)
q3 = animeList['favorite_per_member'].quantile(q=0.75)
# Intervalo interquartil
iqr = q3 - q1
print(iqr)

In [ ]:
animeList[animeList['favorite_per_member'] < (q1-1.5*iqr)]

Apenas os 2 ultimos da lista de favorites_per_member maiores que q3 + 1.5 * iqr são outliers, o restante é normal pela popularidade

In [ ]:
animeList[animeList['favorite_per_member'] > (q3+1.5*iqr)]

In [ ]:
animeList = animeList[animeList["favorite_per_member"] > (q1-1.5*iqr)]
animeList = animeList[animeList["title"] != 'Ultra B']
animeList = animeList[animeList["title"] != 'Zinba']

#### Normalização

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# animeList['scoreNorm'] = (animeList['score'] - animeList['score'].min()) / (animeList['score'].max() - animeList['score'].min())
animeList['scoreNorm'] = scaler.fit_transform(animeList[['score']])
print(animeList['scoreNorm'].describe())
animeList['scoreNorm'].hist()

In [ ]:
# animeList['membersLogNorm'] = (animeList['members_log'] - animeList['members_log'].min()) / (animeList['members_log'].max() - animeList['members_log'].min())
animeList['membersLogNorm'] = scaler.fit_transform(animeList[['members_log']])
print(animeList['membersLogNorm'].describe())
animeList['membersLogNorm'].hist()

Gerando outro dataset normalizado

In [ ]:
animeListNorm = pd.DataFrame()
animeListNorm['title'] = animeList['title']
animeListNorm['members'] = scaler.fit_transform(animeList[['members']])
animeListNorm['favorites'] = scaler.fit_transform(animeList[['favorites']])
animeListNorm['episodes'] = scaler.fit_transform(animeList[['episodes']])
animeListNorm['score'] = animeList['score']
animeListNorm['scoreNorm'] = scaler.fit_transform(animeList[['score']])
animeListNorm.head()

#### Multivariada

Usar o MultiLabelBinarizer para realizar o one hot encoding das listas de generos

In [ ]:
mlb = MultiLabelBinarizer()
tempAnimeList = animeList.copy()
tempAnimeList = tempAnimeList.join(
    pd.DataFrame(mlb.fit_transform(tempAnimeList.pop('genresList')),
                 columns=mlb.classes_,
                 index=tempAnimeList.index))

In [ ]:
tempAnimeList.head()

In [ ]:
# Dropar as colunas que não númericas e as geradas pelo log
tempAnimeList = tempAnimeList.drop(columns=['title', 'studios', 'genres','source', 'studiosList', 'categoryByMembers', 'studio', 'members_log'])
# Atribuir os valores normalizados com o dataset processado acima
tempAnimeList['members'] = animeListNorm['members']
tempAnimeList['favorites'] = animeListNorm['favorites']
tempAnimeList['episodes'] = animeListNorm['episodes']
tempAnimeList['score'] = animeListNorm['scoreNorm']

In [ ]:
clf = IsolationForest(random_state=2020, behaviour="new")
clf.fit(tempAnimeList)

In [ ]:
scores = clf.predict(tempAnimeList)

In [ ]:
scores

In [ ]:
tempAnimeList = animeList.copy()
tempAnimeList['outlier'] = scores
tempAnimeList[tempAnimeList['outlier'] == -1]

In [ ]:
tempAnimeList = tempAnimeList[tempAnimeList['outlier'] != -1]

In [ ]:
print(len(tempAnimeList))
tempAnimeList.head()

Foram 235 outliers detectados, porém este dataset não será utilizado na análise exploratória.

# Amostra de dados e resultados


## Exploratory Data Analysis

### Distribuição dos scores

Pelo gráfico, conseguimos ver a média de 6,68 com desvio padrão de 0,83

In [ ]:
plt.figure(figsize=(10,10))
rating= animeList.score.astype(float)
print(animeList.score.astype(float).mean())
print(animeList.score.astype(float).std())
sns.distplot(rating, bins=20)

### Top 10 animes por quantidade de membros

In [ ]:
mostMembers = animeList.sort_values('members', ascending=False).head(10).set_index('title')
plt.figure(figsize=(15,10))
sns.barplot(mostMembers['members'], mostMembers.index)

O primeiro anime **Death Note** está muito na frente em relação aos demais, principalmente a partir do **Tokyo Ghoul** onde a diferença da margem é mais significativa

### Análise dos Gêneros

In [ ]:
tempGenres = animeList.explode("genresList")
tempGenres['genresList'] = tempGenres['genresList'].astype('category')

In [ ]:
countGenres = tempGenres['genresList'].value_counts()
plt.figure(figsize=(15,10))
sns.barplot(x=countGenres.values, y=countGenres.index, orient="h", order=countGenres.index)
plt.xlabel("Quantity of animes per genre")
plt.show()

O gênero com a maior quantidade de animes é o de **comédia**, seguido de **ação**. Depois a margem aumenta significativamente em relação a **fantasia**.

In [ ]:
meanGenres = tempGenres.groupby("genresList").score.mean().sort_values(ascending=False)
plt.figure(figsize=(15,10))
sns.barplot(x=meanGenres.values, y = meanGenres.index,orient="h", order=meanGenres.index)
plt.xlabel("Mean scores per genres")
plt.show()

Os animes de **thriller** e **mistério** são os com maior nota em média, seguido de **shounen**, este que é um dos gêneros mais famosos do mercado. No final da lista, os animes considerados +18 são os que tem menor nota. 

In [ ]:
medianGenres = tempGenres.groupby("genresList").score.median().sort_values(ascending=False)
plt.figure(figsize=(15,10))
sns.barplot(x=medianGenres.values, y = medianGenres.index,orient="h", order=medianGenres.index)
plt.xlabel("Median scores per genres")
plt.show()

### Análise dos Estúdios

In [ ]:
tempStudios = animeList.explode("studiosList")
tempStudios['studiosList'] = tempStudios['studiosList'].astype('category')

In [ ]:
countStudios = tempStudios['studiosList'].value_counts()[1:50]
plt.figure(figsize=(15,10))
sns.barplot(x=countStudios.values, y=countStudios.index, orient="h", order=countStudios.index)
plt.xlabel("Quantity of animes per Studios")
plt.show()

Os estúdios da **Toei**, **Sunrise** e **Madhouse** são os que mais produziram animes. Isso se deve ao seu tempo no mercado. A **Toei** produz animes desde 1948, a **Sunrise** e a **Madhouse** desde 1972.

In [ ]:
meanStudios = tempStudios.groupby("studiosList").score.mean().sort_values(ascending=False)[:50]
plt.figure(figsize=(15,10))
sns.barplot(x=meanStudios.values, y=meanStudios.index,orient="h", order=meanStudios.index)
plt.xlabel("Mean scores per Studios")
plt.show()

### Distribuição por Origem

In [ ]:
animeList.source.value_counts().plot.pie(autopct="%.0f%%",figsize=(15,10),pctdistance=0.8,
wedgeprops=dict(width=0.4))
plt.show()

Pela distribuição, os animes oriundos de mangás (quadrinhos japoneses) são maioria, seguido por anime de roteiros originais. A grande proporção de Unknown se deve ao fato do MyAnimeList não ter o campo de origem preenchido.

### Relação de episódios por score

In [ ]:
animeList['episodes'].describe()

Plotar os animes com episodios menor-igual a 50, pois é onde se encontra a maioria dos animes. 

In [ ]:
temp = animeList[animeList['episodes'] <= 50]
plt.figure(figsize=(15,10))
sns.set_context('paper')
ax = sns.jointplot(x="score", y="episodes", data = temp, color = 'orange')
ax.set_axis_labels("Score", "Episódios")

Pelo gráfico, não é possivel visualizar uma correlação entre a quantidade de episodios para uma nota positiva. Para notas ruins (<= 4), pode ser observado que a maioria possuem apenas 1 episódio, sendo muito provavelmente curta-metragens, clipes ou comerciais.

Também é possivel detectar picos próximos de 12 e 24 episódios. Isso se deve porque a maioria dos animes são lançados por temporada de 3 ou 6 meses semanalmente, resultando em 12 ou 24 episódios.

#### Relação do score pelos quantis dos membros

In [ ]:
meanCategory = tempStudios.groupby("categoryByMembers").score.mean().sort_values(ascending=False)
plt.figure(figsize=(15,10))
sns.barplot(x=meanCategory.values, y=meanCategory.index,orient="h", order=meanCategory.index)
plt.xlabel("Mean scores per category")
plt.show()

Pelo gráfico acima das categorias de quantidade de membros divididos em 6 quantis, é possivel visualizar que a média do score sobe a medida que a quantidade de membros é maior. Isso poderá ser observado também nos gráficos de correlação abaixo.

## Covariancias e correlações

In [ ]:
elements = ['score', 'members', 'favorites', 'episodes','favorite_per_member']

### Matriz de dispersão

In [ ]:
pd.plotting.scatter_matrix(animeList[elements], figsize=(15, 10));

Só pela matriz de dispersão, é possivel notar que a razão de favoritos por membros aparenta crescer "linearmente" em relação ao score

### Pearson

In [ ]:
figure = plt.figure(figsize=(15,10))
sns.heatmap(animeList.corr(method='pearson'), annot=True)
plt.show()

### Spearman

In [ ]:
figure = plt.figure(figsize=(15,10))
sns.heatmap(animeList.corr(method='spearman'), annot=True)
plt.show()

A partir das correlações apresentadas, vemos que:
- A quantidade de favoritos varia de acordo com a quantidade de membros
- O Score é influenciado pela quantidade de favoritos
- Pouco importa a quantidade de episódios pra determinar o score

### Covariância

In [ ]:
figure = plt.figure(figsize=(15,10))
sns.heatmap(animeList[elements].cov(), annot=True)
plt.show()
animeList.cov().head()

In [ ]:
animeList.head()

# Testes de normalidade e hipótese

In [ ]:
score_notna = animeList[animeList['score'].notna()].score
members_notna = animeList[animeList['members'].notna()].members
favorites_notna = animeList[animeList['favorites'].notna()].favorites

In [ ]:
def generate_anderson(data):
    result = stats.anderson(data)
    print('Statistic: %.3f' % result.statistic)
    p = 0
    for i in range(len(result.critical_values)):
      sl, cv = result.significance_level[i], result.critical_values[i]
      if result.statistic < result.critical_values[i]:
        print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
      else:
        print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))

A análise por Anderson se viu necessária para verificação de normalidade, pela grande quantidade de dados presente na coleta, pois análise por Shapiro demonstra ser imprecisa para 
N > 5000

In [ ]:
animeList.score.hist()
generate_anderson(animeList.score)

Mesmo que o score aparenta ter um histograma com uma curva de Gauss, pelo teste de Anderson, a hipotese nula foi rejeitada.

In [ ]:
animeList.plot.scatter(x='members',y='score')

In [ ]:
members_notna.hist()
generate_anderson(members_notna)

In [ ]:
favorites_notna.hist()
generate_anderson(favorites_notna)

In [ ]:
animeList.members.hist()
generate_anderson(animeList.members)

Nenhum dos dados seguem uma normal, logo o teste de hipotese a ser utilizado seria o mann-whitney, pois a comparação seria entre membros não-pariados e não-paremétricos

### Analise Comédia e Ação (Mais quantitativos)

In [ ]:
# Amostra de dados que são de Comédia, mas não de ação.
animeList[~animeList['genres'].str.contains('Action') & animeList['genres'].str.contains('Comedy')].score.hist()
animeList[~animeList['genres'].str.contains('Action') & animeList['genres'].str.contains('Comedy')].describe()

In [ ]:
# Melhor anime cotado somente de Comedia
animeList[(~animeList['genres'].str.contains('Action') & animeList['genres'].str.contains('Comedy')) & (animeList.score == 8.96)]

In [ ]:
# Pior anime cotado somente de Comedia
animeList[(~animeList['genres'].str.contains('Action') & animeList['genres'].str.contains('Comedy')) & (animeList.score == 2.04)]

In [ ]:
# Amostra de dados que são de Ação, mas não de comédia.
animeList[animeList['genres'].str.contains('Action') & ~animeList['genres'].str.contains('Comedy')].score.hist()
animeList[animeList['genres'].str.contains('Action') & ~animeList['genres'].str.contains('Comedy')].describe()

In [ ]:
# Melhor anime cotado somente de Ação
animeList[(animeList['genres'].str.contains('Action') & ~animeList['genres'].str.contains('Comedy')) & (animeList.score == 9.12)]

In [ ]:
# Pior anime cotado somente de Ação
animeList[(animeList['genres'].str.contains('Action') & ~animeList['genres'].str.contains('Comedy')) & (animeList.score == 3.16)]

In [ ]:
# Diferença de descrição
animeList[animeList['genres'].str.contains('Action') & ~animeList['genres'].str.contains('Comedy')].describe() - animeList[~animeList['genres'].str.contains('Action') & animeList['genres'].str.contains('Comedy')].describe()

In [ ]:
animeList[~animeList['genres'].str.contains('Action') & animeList['genres'].str.contains('Comedy')].describe()/animeList[animeList['genres'].str.contains('Action') & ~animeList['genres'].str.contains('Comedy')].describe()

In [ ]:
# Score
stats.mannwhitneyu(animeList[~animeList['genres'].str.contains('Action') & animeList['genres'].str.contains('Comedy')].score,animeList[~animeList['genres'].str.contains('Comedy') & animeList['genres'].str.contains('Action')].score)

In [ ]:
# Members
stats.mannwhitneyu(animeList[~animeList['genres'].str.contains('Action') & animeList['genres'].str.contains('Comedy')].members,animeList[~animeList['genres'].str.contains('Comedy') & animeList['genres'].str.contains('Action')].members)

A partir da comparação dos 2 gêneros mais populares, vemos que:
- A diferença entre scores não são tão absurdas
- Em mediana, animes de ação que não são comédia possui nota ligeiramente menor
- Animes de ação normalmente possuem uma comunidade maior, tanto em membros quanto em favoritos.
- Pelo teste de hipotese, vemos que os dados não são iguais, tendo o pvalue um valor pequeno

### Analise Shounen e Mistério (Maiores médias)

In [ ]:
# Amostra de dados do que são de Mistério, mas não de Shounen (Ação para garotos adolescentes)
animeList[~animeList['genres'].str.contains('Shounen') & animeList['genres'].str.contains('Mystery')].score.hist()
animeList[~animeList['genres'].str.contains('Shounen') & animeList['genres'].str.contains('Mystery')].describe()

In [ ]:
# Melhor anime cotado somente de Mistério
animeList[(~animeList['genres'].str.contains('Shounen') & animeList['genres'].str.contains('Mystery')) & (animeList.score == 8.93)]

In [ ]:
# Pior anime cotado somente de Mistério
animeList[(~animeList['genres'].str.contains('Shounen') & animeList['genres'].str.contains('Mystery')) & (animeList.score == 3.40)]

In [ ]:
# Amostra de dados do que são de 'Ação para adolescentes masculinos', mas não de Mistério
animeList[animeList['genres'].str.contains('Shounen') & ~animeList['genres'].str.contains('Mystery')].score.hist()
animeList[(animeList['genres'].str.contains('Shounen') & ~animeList['genres'].str.contains('Mystery'))].describe()

In [ ]:
# Melhor anime cotado somente de Luta
animeList[(animeList['genres'].str.contains('Shounen') & ~animeList['genres'].str.contains('Mystery')) & (animeList.score == 9.22)]

In [ ]:
# Pior anime cotado somente de Luta
animeList[(animeList['genres'].str.contains('Shounen') & ~animeList['genres'].str.contains('Mystery')) & (animeList.score == 3.52)]

In [ ]:
animeList[animeList['genres'].str.contains('Shounen') & ~animeList['genres'].str.contains('Mystery')].describe() - animeList[~animeList['genres'].str.contains('Shounen') & animeList['genres'].str.contains('Mystery')].describe()

In [ ]:
# Porcentagens
animeList[animeList['genres'].str.contains('Shounen') & ~animeList['genres'].str.contains('Mystery')].describe()/animeList[~animeList['genres'].str.contains('Shounen') & animeList['genres'].str.contains('Mystery')].describe()

In [ ]:
# Score
stats.mannwhitneyu(animeList[~animeList['genres'].str.contains('Shounen') & animeList['genres'].str.contains('Mystery')].score,animeList[~animeList['genres'].str.contains('Mystery') & animeList['genres'].str.contains('Shounen')].score)

In [ ]:
# Members
stats.mannwhitneyu(animeList[~animeList['genres'].str.contains('Shounen') & animeList['genres'].str.contains('Mystery')].members,animeList[~animeList['genres'].str.contains('Mystery') & animeList['genres'].str.contains('Shounen')].members)

A partir da comparação dos 2° e 3° gêneros de maior media e mediana vemos que:
- Em maioria, o público prefere animes de Mistério que não são Shounen.
- Comunidades de animes de Mistério normalmente são maiores.
- Animes de mistério normalmente possuem score maior.
- Pelo teste de hipotese, vemos que os dados não são iguais, pois o p-value teve um valor pequeno, sendo a hipotese nula rejeitada

### Analise Egg Firm e Studio Chizu (Estúdios com as maiores médias)

In [ ]:
animeList[animeList['studios'].str.contains('Egg Firm')].describe()

In [ ]:
animeList[animeList['studios'].str.contains('Egg Firm')]

In [ ]:
animeList[animeList['studio'] == 'Studio Chizu'].describe()

In [ ]:
animeList[animeList['studio'] == 'Studio Chizu']

In [ ]:
animeList[animeList['genres'].str.contains('Comedy') & animeList['genres'].str.contains('Supernatural')].score.median()

In [ ]:
#Score
stats.mannwhitneyu(animeList[animeList['studio'] == 'Studio Chizu'].score,animeList[animeList['studios'].str.contains('Egg Firm')].score)

In [ ]:
#Membros
stats.mannwhitneyu(animeList[animeList['studio'] == 'Studio Chizu'].members,animeList[animeList['studios'].str.contains('Egg Firm')].members)

Para os estudios de maior média, não há tanto a se comparar, pela quantidade de animes criados por cada um, porém, é notável que:
- **Studio Chizu** normalmente é um estudio de filmes (Só realizou filmes até então)
- **Egg Firm**, juntamente a **J.C. Staff** produziram Saiki no Kusuo, um famoso anime lançado em 28 de Dezembro  de 2018 com maior foco em comédia e sobrenatural (poderes psiquicos), generos dos quais possuem uma média de score alta.
- **Studio Chizu** em maioria realiza filmes de Fantasia, onde 'Bakemono no Ko' é o único que não possui o gênero de fantasia no MyAnimeList, porém, ao nosso ver, pode ser considerado também deste genero, tendo em vista que se trata de um garoto que vai para um mundo onde há somente monstros.

### Amostra de dados quantitativos de gêneros de Toei Animation, Sunrise e Madhouse

In [ ]:
# Quantidade de animes feitos por gênero pela 'Toei Animation'
tempGenres = animeList[(animeList['studio'] == 'Toei Animation')].explode("genresList")
tempGenres['genresList'] = tempGenres['genresList'].astype('category')
tempGenres['genresList'].value_counts()[1:10]

In [ ]:
# Quantidade de animes feitos por gênero pela 'Sunrise'
tempGenres = animeList[(animeList['studio'] == 'Sunrise')].explode("genresList")
tempGenres['genresList'] = tempGenres['genresList'].astype('category')
tempGenres['genresList'].value_counts()[1:10]

In [ ]:
# Quantidade de animes feitos por gênero pela 'Madhouse'
tempGenres = animeList[(animeList['studio'] == 'Madhouse')].explode("genresList")
tempGenres['genresList'] = tempGenres['genresList'].astype('category')
tempGenres['genresList'].value_counts()[1:10]

Com o top 10 de animes mais feitos pelos estúdios 'Toei Animation','Sunrise' e 'Madhouse', temos alguns gêneros em comum como:
- Action    :[224,213,106]
- Comedy    :[207,110,120]
- Shounen   :[201,87  ,53]

### Analise Toei Animation e Sunrise

In [ ]:
# Toei e Sunrise
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Action'))].score,animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Action'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Action'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Action'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Action'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Action'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Action'))].describe() - animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Action'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Action'))].describe()/animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Action'))].describe()

Para animes de Ação, a **Sunrise** possui vantagem de score em todos os pontos, além de possuir comunidade ligeiramente maior no 1° e 3° quartil (20% e 9% respectivamente)

In [ ]:
# Toei e Sunrise
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Comedy'))].score,animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Comedy'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Comedy'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Comedy'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Comedy'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Comedy'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Comedy'))].describe() - animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Comedy'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Comedy'))].describe()/animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Comedy'))].describe()

Para animes de comédia, a **Sunrise** possui grande vantagem de score em quase todos os pontos, porém sua comunidade é menor, tendo em vista a menor quantidade de membros para 2° e o 3° quartil (46% e 21% respectivamente)

In [ ]:
# Toei e Sunrise
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Shounen'))].score,animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Shounen'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Shounen'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Shounen'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Shounen'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Shounen'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Shounen'))].describe() - animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Shounen'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Shounen'))].describe()/animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Shounen'))].describe()

Para animes de Shounen, a **Sunrise** possui vantagem de score em todos os quartis, porém, novamente sua comunidade é menor, tendo em vista a menor quantidade de membros para 1°, 2° e o 3° quartil (57%, 68% e 40%)

Desta forma temos que
- De forma geral, Sunrise tem maior "qualidade" entre os animes
- Animes da **Toei Animation** alcança um publico maior
- Animes de ação da **Sunrise** são mais propensos a sucesso, tanto em qualidade quanto em alcance da comunidade
- Dados não são identicos, tendo em vista que nenhum teste normal possuiu p-value alto

### Analise Toei Animation e Madhouse

In [ ]:
# Toei e Madhouse
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Action'))].score,animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Action'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Action'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Action'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Action'))].describe() - animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Action'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Action'))].describe()/animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Action'))].describe()

Animes de ação da **Toei Animation** normalmente possui score ligeiramente menor em relação a **Madhouse**, e comunidade menor em todos os quartis (60%, 58% e 65%)

In [ ]:
# Toei e Madhouse
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Comedy'))].score,animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Comedy'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Comedy'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Comedy'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Comedy'))].describe() - animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Comedy'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Comedy'))].describe()/animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Comedy'))].describe()

Os animes da **Toei Animation** de comédia possuem scores ligeiramente menores em relação a **Madhouse**, porém, em comunidade é menor em seus quartis (33%, 8%, 36%)

In [ ]:
# Toei e Madhouse
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Shounen'))].score,animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Shounen'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Shounen'))].score.hist()

In [ ]:
animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Shounen'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Shounen'))].describe() - animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Shounen'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation') & (animeList['genres'].str.contains('Shounen'))].describe()/animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Shounen'))].describe()

Para animes do gênero de Shounen, **Toei Animation** possui scores ligeiramente menores, além de uma comunidade menor em todos os quartis (53%, 55%, 45%)

Temos então na comparação da **Toei Animation** com a **Madhouse** que:
- Animes da **Toei Animation** geralmente possuem score ligeiramente menor
- O alcance dos animes da **Madhouse**,em sua maioria, é maior, tendo em vista uma quantidade de membros maior.
- Os dados não são identicos por possuir p-value muito baixo

### Analise Sunrise e Madhouse

In [ ]:
# Sunrise e Madhouse
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Action'))].score,animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Action'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Action'))].describe()-animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Action'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Action'))].describe()/animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Action'))].describe()

Em relação aos animes de Ação, a **Sunrise** possui scores ligeiramente maiores aos da **Madhouse**, porém, sua comunidade é bem menor em relação aos da 'Madhouse', tendo em seus quartis uma comunidade (40%, 64%, 61%) maior.

In [ ]:
# Sunrise e Madhouse
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Comedy'))].score,animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Comedy'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Comedy'))].describe()-animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Comedy'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Comedy'))].describe()/animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Comedy'))].describe()

Em relação a Comédia, novamente, animes da **Sunrise** possuem score ligeiramente maior em relação a **Madhouse**, e uma comunidade menor em seus quartis (28%, 50%, 50%)

In [ ]:
# Sunrise e Madhouse
stats.mannwhitneyu(animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Shounen'))].score,animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Shounen'))].score)

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Shounen'))].describe()-animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Shounen'))].describe()

In [ ]:
animeList[(animeList['studio'] == 'Sunrise') & (animeList['genres'].str.contains('Shounen'))].describe()/animeList[(animeList['studio'] == 'Madhouse') & (animeList['genres'].str.contains('Shounen'))].describe()

Para animes de Shounen, temos novamente a **Sunrise** como animes de maior score, e a **Madhouse** com a comunidade muito maior em seus quartis (80%, 86%, 67%)
Realizando uma pequena pesquisa a **Madhouse** realizou a animação de animes muito famosos como:
- Hunter X Hunter
- Overlord
- One Punch Man
- No game no life

Talvez desta forma as pessoas venham a se interessar para ir mais a fundo em produções de gênero parecido deste estúdio, resultando sua grande popularidade em animes distintos.

## Score Plot dos estúdios analisados

In [ ]:
animeList[(animeList['studio'] == 'Sunrise')].score.plot.box()

In [ ]:
animeList[(animeList['studio'] == 'Madhouse')].score.plot.box()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation')].score.plot.box()

### Members Plot dos estúdios analisados

In [ ]:
animeList[(animeList['studio'] == 'Sunrise')].membersLogNorm.plot.box()

In [ ]:
animeList[(animeList['studio'] == 'Madhouse')].membersLogNorm.plot.box()

In [ ]:
animeList[(animeList['studio'] == 'Toei Animation')].membersLogNorm.plot.box()

Temos então na comparação da **Sunrise** com a **Madhouse** que:
- Animes da 'Sunrise' geralmente possuem score ligeiramente maior
- O alcance dos animes da **Madhouse**,em sua maioria, são maiores, principalmente de Shounen, tendo em vista uma quantidade de membros maior.
- Os dados não são identicos por possuir p-value muito baixo

Como análise geral, temos que:
- Animes da **Madhouse** possuem tendência a possuir um alcance muito maior, principalmente para animes Shounen.
- Animes da **Sunrise** possui scores ligeiramente maiores em relação aos demais estúdios.
- Animes da **Toei Animation**, apesar de possuirem quantidade de membros menores, prossegue sendo boa escolha por não haver tanta diferença em seus scores.

**Toei Animation** é responsável pela produção de alguns animes longos famosos como One Piece, Cavaleiros do Zodiaco e Dragon Ball Z

# Clustering

Acima, dividimos os membros em 6 quantis, e também pelos gráficos conseguimos observar que os animes com mais membros tendem a ter um score maior. Agora, ao invés de separar pelos quantis, tentaremos achar clusters.

In [ ]:
animeCluster = animeList[['score', 'members']]
# toCluster = np.asarray([np.asarray(c['score']), np.asarray(c['members'])]).T

Utilizaremos o silhouette score para selecionar qual o melhor número de clusters, variando de 2 a 6

In [ ]:
X = animeCluster
for k in tqdm(range(2,7)):
    k_means = KMeans(n_clusters = k, random_state=2020)
    labels = k_means.fit_predict(X)
    silhouette_avg = silhouette_score(X, labels)
    print("For k clusters =", k,
          "The average silhouette_score is :", silhouette_avg)


k = 2 obteve o melhor score, porém utilizaremos k = 3 para ter uma divisão maior

In [ ]:
k_means = KMeans(n_clusters=3, random_state=2020)

labels = k_means.fit_predict(animeCluster)
animeCluster['label'] = labels
animeCluster['label'] = animeCluster['label'].astype('category')

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(animeCluster['score'], animeCluster['members'], c=labels)
plt.xlabel('3 clusters de score e a razão de favoritos com membros');

In [ ]:
countCluster = animeCluster.label.value_counts()
plt.figure(figsize=(15,10))
sns.barplot(x=countCluster.values, y=countCluster.index,orient="h", order=countCluster.index)
plt.xlabel("Qt per cluster")
plt.show()

In [ ]:
meanCluster = animeCluster.groupby("label").score.mean().sort_values(ascending=False)
plt.figure(figsize=(15,10))
sns.barplot(x=meanCluster.values, y=meanCluster.index,orient="h", order=meanCluster.index)
plt.xlabel("Mean scores per cluster")
plt.show()

In [ ]:
meanCluster = animeCluster.groupby("label").members.mean().sort_values(ascending=False)
plt.figure(figsize=(15,10))
sns.barplot(x=meanCluster.values, y=meanCluster.index,orient="h", order=meanCluster.index)
plt.xlabel("Mean members per cluster")
plt.show()

Com separacao do dataset com 3 clusters, foi dividido os animes com maior nota dos com menor nota, sendo também verdade para a média da quantidade de membros.